<h1>Warning</h1>
This seems to not be processing The Moldau... need to figure this out before running again

In [2]:
# features_pipeline.py
import os
import json
import pandas as pd
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
from sklearn.preprocessing import StandardScaler
import multiprocessing

from features_worker import process_file

INPUT_DIR = 'Musicmap_Improved_Download'
OUTPUT_DIR = '15_second_features_augmented/musicmap_processed_output'
SEGMENT_LENGTH = 15  # seconds

FEATURE_TYPES = {
#    'mel_db_rgb': 'mel_db_rgb',
    'mel_db_gray': 'mel_db_gray',
#    'mel_pcen_rgb': 'mel_pcen_rgb',
    'mel_pcen_gray': 'mel_pcen_gray',
#    'mfcc_rgb': 'mfcc_rgb',
    'mfcc_gray': 'mfcc_gray',
#    'chroma_rgb': 'chroma_rgb',
    'chroma_gray': 'chroma_gray',
#    'chroma_cq_rgb': 'chroma_cq_rgb',
    'chroma_cq_gray': 'chroma_cq_gray',
#    'chroma_bs_rgb': 'chroma_bs_rgb',
    'chroma_bs_gray': 'chroma_bs_gray',
    'mfcc_plot': 'mfcc_plot',
    'hpss_median': 'hpss_median',
    'hpss_mean': 'hpss_mean',
#    'resnet_mel_rgb': 'resnet_mel_rgb'
}

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

def main():
    all_files = [str(p) for p in Path(INPUT_DIR).rglob('*.m4a')]
    args_list = [(fp, FEATURE_TYPES, SEGMENT_LENGTH, OUTPUT_DIR) for fp in all_files]

    all_features = []
    durations = []
    with ProcessPoolExecutor() as executor:
        #for feats, duration_info in executor.map(process_file, all_files):
        for feats, duration_info in executor.map(process_file, args_list):
            all_features.extend(feats)
            durations.append(duration_info)
#    with ProcessPoolExecutor() as executor:
#        for result in executor.map(process_file, args_list):
#            all_features.extend(result)

    df = pd.DataFrame(all_features)
    csv_dir = os.path.join(OUTPUT_DIR, 'features_csv')
    ensure_dir(csv_dir)

    # Save raw CSVs
    for col in df.columns:
        if col not in ['file', 'segment', 'genre_id']:
            df[['file', 'segment', 'genre_id', col]].to_csv(os.path.join(csv_dir, f'{col}.csv'), index=False)

    # Normalize and save scaler
    feature_cols = [col for col in df.columns if col not in ['file', 'segment', 'genre_id']]
    scaler = StandardScaler()
    norm_data = scaler.fit_transform(df[feature_cols])
    norm_df = pd.DataFrame(norm_data, columns=feature_cols)
    norm_df[['file', 'segment', 'genre_id']] = df[['file', 'segment', 'genre_id']]
    norm_df.to_csv(os.path.join(csv_dir, 'features_normalized.csv'), index=False)

    with open(os.path.join(csv_dir, 'scaler.json'), 'w') as f:
        json.dump({
            'mean': scaler.mean_.tolist(),
            'scale': scaler.scale_.tolist(),
            'features': feature_cols
        }, f, indent=2)
    
    duration_df = pd.DataFrame(durations)
    duration_df.to_csv(os.path.join(csv_dir, 'song_durations.csv'), index=False)


if __name__ == '__main__':
    multiprocessing.set_start_method("spawn", force=True)
    main()


/opt/homebrew/anaconda3/envs/deep-learning/lib/python3.12/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/opt/homebrew/anaconda3/envs/deep-learning/lib/python3.12/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/Volumes/Sonnet_SSD/DocumentsS/GitHub/neuromatch_spectrograms/features_worker.py:44: RuntimeWarning: invalid value encountered in divide
  norm = (data - np.min(data)) / (np.max(data) - np.min(data))
/opt/homebrew/anaconda3/envs/deep-learning/lib/python3.12/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
/Volumes/Sonnet_SSD/DocumentsS/GitHub/neuromatch_spectrograms/features_worker.py:40: RuntimeWarning: invalid value encountered in divide
  norm = 255 * (data - np.min(data)) / (np.max(data) - np.min(data))
/Volumes/Sonnet_SSD/DocumentsS/Gi